In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
import time

# Set up WebDriver
driver = webdriver.Chrome()
url = "https://web.archive.org/web/20211229085102/https://www2.hse.ie/services/covid-19-vaccination-centres/"
driver.get(url)

# Output file
output_file = "vaccination_centres.csv"

try:
    # Wait for the dropdown to load (wait until more than just the placeholder is present)
    WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.ID, "service_area"))
    )

    # Scroll the page to ensure content is loaded
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Allow time for dynamic content to load after scrolling

    dropdown = driver.find_element(By.ID, "service_area")
    options = dropdown.find_elements(By.TAG_NAME, "option")

    # Debugging: Print all the options in the dropdown
    print("Dropdown options:")
    for option in options:
        print(option.text)

    if len(options) < 2:
        print("The dropdown did not load correctly.")
        driver.quit()
        exit()

    # Output file
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        # Write CSV header
        writer.writerow(["County", "Centre Name", "Address"])

        # Skip the first option (placeholder) and iterate through counties
        for option in options[1:]:
            try:
                county = option.text
                print(f"Processing county: {county}")
                option.click()

                # Trigger a change event (if needed)
                driver.execute_script("document.getElementById('service_area').dispatchEvent(new Event('change'));")

                # Wait for vaccination centres to load
                WebDriverWait(driver, 10).until(
                    EC.presence_of_all_elements_located((By.CLASS_NAME, "hse-u-margin-bottom-4"))
                )

                # Check if any centres exist
                centres = driver.find_elements(By.CLASS_NAME, "hse-u-margin-bottom-4")
                print(f"Found {len(centres)} centres for county: {county}")

                if not centres:
                    print(f"No centres found for county: {county}")
                    continue

                for centre in centres:
                    try:
                        # Extract centre name (h3 tag)
                        name_elem = centre.find_element(By.TAG_NAME, "h3")  # Centre name is in <h3> tag
                        name = name_elem.text.strip() if name_elem else "N/A"
                        print(f"Name found: {name}")

                        # Extract address (p tag right after the h3 tag)
                        address_elem = centre.find_element(By.XPATH, ".//following-sibling::p")  # Address is in <p> tag after <h3>
                        address = address_elem.text.strip() if address_elem else "N/A"
                        print(f"Address found: {address}")

                        # Write to CSV if both are found
                        writer.writerow([county, name, address])
                        print(f"Processed centre: {name}, {address}")

                    except Exception as centre_error:
                        print(f"Error processing centre: {centre_error}")
                        continue

            except Exception as county_error:
                print(f"Error processing county {county}: {county_error}")
                continue

    print("Scraping completed successfully!")

except Exception as e:
    print(f"An unexpected error occurred: {e}")

finally:
    driver.quit()


Dropdown options:
Please select
Carlow
Cavan
Clare
Cork
Donegal
Dublin
Galway
Kerry
Kildare
Kilkenny
Laois
Leitrim
Limerick
Longford
Louth
Mayo
Meath
Monaghan
Offaly
Roscommon
Sligo
Tipperary
Waterford
Westmeath
Wexford
Wicklow
Processing county: Carlow
Found 2 centres for county: Carlow
Name found: Carlow Vaccination Centre
Address found: Woodford Dolmen Hotel, Kilkenny Road, Carlow, R93 N207
Processed centre: Carlow Vaccination Centre, Woodford Dolmen Hotel, Kilkenny Road, Carlow, R93 N207
Error processing centre: Message: no such element: Unable to locate element: {"method":"tag name","selector":"h3"}
  (Session info: chrome=131.0.6778.206); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF794E680D5+2992373]
	(No symbol) [0x00007FF794AFBFD0]
	(No symbol) [0x00007FF79499590A]
	(No symbol) [0x00007FF7949E926E]
	(No symbol) [0x00007FF7949E955C]
	(No s